### Problem 1. (c)

In [1]:
using JuMP
using Graphs #, SimpleWeightedGraphs
#using LightGraphs
#using DataStructures
using Optim #, LineSearches
using DataFrames

In [2]:
function FindSPP(dest, dijkalgorithm)
    # Shortest path of OD pair
    p = []
    push!(p, dest)
    dest = dijkalgorithm.parents[dest]
    while dest != 0 
        push!(p, dest)
        dest = dijkalgorithm.parents[dest]
    end
    return reverse(p)
end

FindSPP (generic function with 1 method)

In [13]:
function SPP(A,B,K,v_bar, graph, distmatrix, origin)
    c = B ./ (K - v_bar); replace!(c, NaN => 0); c = A + c
    
    #Dijkstra algorithm
    dijk = dijkstra_shortest_paths(graph, o, distmatrix)

    # Shortest path of OD pair
    path = [FindSPP(3, dijk), FindSPP(4, dijk)]
    return path
end

SPP (generic function with 2 methods)

In [5]:
function J(h_bar, d_bar, θ)
    h_temp = h_bar + θ*d_bar
    v_temp = [h_temp[1] + h_temp[2] + h_temp[3],
              h_temp[1],
              h_temp[2] + h_temp[3],
              h_temp[2]]
    J = A[1] * (v_temp[1]) + A[2] * (v_temp[2]) + (B[2] / (K[2] - v_temp[2])) * (v_temp[2]) + A[3] * (v_temp[3]) + (B[3] / (K[3] - v_temp[3])) * (v_temp[3]) + A[4] * (v_temp[4]) + (B[4] / (K[4] - v_temp[4] )) * (v_temp[4])
    return J
  end

J (generic function with 1 method)

In [6]:
# Input
path = [[1,2,3], [1,2,4,3], [1,2,4]]

start_node = [1,2,2,4]
end_node = [2,3,4,3]

# initial feasible solution
Q = [60, 40]

A = [10, 10, 100, 100]
B = [0, 50, 50, 50]
K = [100, 50, 100, 100]

h_bar = [0, 60, 40] # initial h
#h_bar = [30, 30, 40.0]
v_bar= [h_bar[1] + h_bar[2] + h_bar[3],
        h_bar[1],
        h_bar[2] + h_bar[3],
        h_bar[2]]

c = B ./ (K - v_bar); replace!(c, NaN => 0); c = A + c

4-element Vector{Float64}:
  10.0
  11.0
  Inf
 101.25

In [11]:
# o,d-pairs
o = 1; d = [3, 4]

# Construct Graph g
no_node = max( maximum(start_node), maximum(end_node) )
no_link = length(start_node)

g = SimpleDiGraph(no_node)
distmx = Inf*ones(no_node, no_node)

for i in 1:no_link
    add_edge!(g, start_node[i], end_node[i])
    distmx[start_node[i], end_node[i]] = c[i]
end

In [14]:
res_h = []; res_v = []; res_θ = []; res_d = []
θ_opt = 0

# Initialization 
k = 0
while k <= 10
    # All or nothing assignment
    p = SPP(A,B,K,v_bar, g, distmx, o) #

    h_new = [0,0,0]
    for i in 1:length(h_new)
        for j in 1:length(p)
            if path[i] == p[j]
                h_new[i] = Q[j]
            end
        end
    end
    # Line search
    d_bar = h_new - h_bar # Determine the direction vector

    opt = Optim.optimize( θ -> J(h_bar, d_bar, θ), 0.0, 1.0, GoldenSection()) #
    if θ_opt == round(Optim.minimizer(opt); digits=3)
        break
    end
    θ_opt = round(Optim.minimizer(opt); digits=3)

    # Update
    h_bar = h_bar + θ_opt * d_bar
    v_bar= [h_bar[1] + h_bar[2] + h_bar[3],
            h_bar[1],
            h_bar[2] + h_bar[3],
            h_bar[2]]

    push!(res_d, d_bar)
    push!(res_θ, θ_opt)
    push!(res_h, h_bar)
    push!(res_v, v_bar)
    k = k + 1
end

df = DataFrame(;k = 1:k, h = res_h, v = res_v, d = res_d, θ = res_θ)

Row,k,h,v,d,θ
,Int64,Any,Any,Any,Any
1,1,"[49.98, 10.02, 40.0]","[100.0, 49.98, 50.02, 10.02]","[60, -60, 0]",0.833
2,2,"[60.0, 0.0, 40.0]","[100.0, 60.0, 40.0, 0.0]","[10.02, -10.02, 0.0]",1.0
3,3,"[60.0, 0.0, 40.0]","[100.0, 60.0, 40.0, 0.0]","[0.0, 0.0, 0.0]",0.382


Row,k,h,v,d,θ
,Int64,Any,Any,Any,Any
1,1,"[49.98, 10.02, 40.0]","[100.0, 49.98, 50.02, 10.02]","[60, -60, 0]",0.833
2,2,"[60.0, 0.0, 40.0]","[100.0, 60.0, 40.0, 0.0]","[10.02, -10.02, 0.0]",1.0
3,3,"[60.0, 0.0, 40.0]","[100.0, 60.0, 40.0, 0.0]","[0.0, 0.0, 0.0]",0.382
